# Intersections
Author: Ida Thrane (idth@itu.dk)

Finding intersections between roads and spatial footprint layers

In [2]:
#Import libraries
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import pandas as pd
from shapely import wkt
import osmnx as ox
import numpy as np
#import dask_geopandas as dask_gpd
import shapely as shp

In [3]:
#G = ox.graph_from_place('Copenhagen Municipality, Denmark')

#Save graph for later use 
#ox.save_graphml(G, "graphs/cph_graph.graphml")

#Load graph
G = ox.load_graphml("graphs/cph_graph.graphml")

In [4]:
#Create node and edges of Copenhagen street network
gdf_nodes, gdf_edges = ox.graph_to_gdfs(
        G,nodes=True, edges=True,
        node_geometry=True,
        fill_edge_geometry=True)

In [5]:
#Define buffer size
buffer_size = 0.1 #10 meter buffer

In [6]:
#Create road polygons
road_polygons = []

#crs set to 32632 which covers Copenhagen and has its units in metres
gdf_edges = gdf_edges.to_crs("EPSG:32632")
for i in gdf_edges["geometry"]:
    #Create polygon around street with the defined buffer size
    polygon = i.buffer(buffer_size)
    road_polygons.append(polygon)

In [7]:
gdf_edges["geometry"] = road_polygons

In [8]:
gdf_edges = gdf_edges.reset_index(level=[0,1])

In [20]:
gdf_edges.columns

Index(['key', 'u', 'v', 'osmid', 'oneway', 'name', 'highway', 'maxspeed',
       'reversed', 'length', 'lanes', 'geometry', 'ref', 'service', 'width',
       'access', 'bridge', 'tunnel', 'junction'],
      dtype='object')

In [23]:
#Read in intersections
#gdf_intersections = gdf_edges.to_csv("graphs/gdf_intersections.csv")

gdf_edges = pd.read_csv("graphs/gdf_intersections.csv")

#Create gdf
gdf_edges['geometry'] = gdf_edges['geometry'].apply(wkt.loads)
gdf_edges = gpd.GeoDataFrame(gdf_edges, crs="EPSG:32632", geometry="geometry")

In [24]:
gdf_roads_short = gdf_edges[['osmid', 'geometry','u','v']]

In [25]:
gdf_roads_short.head()

,osmid,geometry,u,v
0,27226011,"POLYGON ((724031.978 6175537.631, 724031.982 6...",118725,2512504197
1,140412993,"POLYGON ((724051.903 6175566.892, 724059.863 6...",118725,6357644306
2,2371125,"POLYGON ((723797.150 6174181.458, 723786.227 6...",118730,118751
3,25996200,"POLYGON ((724002.014 6174333.243, 724002.020 6...",118732,6534097568
4,683463579,"POLYGON ((724005.826 6174341.896, 724005.834 6...",118732,283603631


In [26]:
gdf_roads_short.crs

<Derived Projected CRS: EPSG:32632>
Name: WGS 84 / UTM zone 32N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 6°E and 12°E, northern hemisphere between equator and 84°N, onshore and offshore. Algeria. Austria. Cameroon. Denmark. Equatorial Guinea. France. Gabon. Germany. Italy. Libya. Liechtenstein. Monaco. Netherlands. Niger. Nigeria. Norway. Sao Tome and Principe. Svalbard. Sweden. Switzerland. Tunisia. Vatican City State.
- bounds: (6.0, 0.0, 12.0, 84.0)
Coordinate Operation:
- name: UTM zone 32N
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [27]:
#Read in shapes and points from Copenhagen retrieved from OSM via Geofrabrik
#shapes
gdf_building_a = gpd.read_file("shapes/building_a.shp")
gdf_landuse_a = gpd.read_file("shapes/landuse_a.shp")
gdf_natural_a = gpd.read_file("shapes/natural_a.shp")
gdf_pofw_a = gpd.read_file("shapes/pofw_a.shp")
gdf_pois_a = gpd.read_file("shapes/pois_a.shp")
gdf_traffic_a = gpd.read_file("shapes/traffic_a.shp")
gdf_water_a = gpd.read_file("shapes/water_a.shp")

#points
gdf_natural_p = gpd.read_file("shapes/natural.shp")
gdf_places_p = gpd.read_file("shapes/places.shp")
gdf_pofw_p = gpd.read_file("shapes/pofw.shp")
gdf_pois_p = gpd.read_file("shapes/pois.shp")
gdf_railways_p = gpd.read_file("shapes/railways.shp")
gdf_roads_p = gpd.read_file("shapes/roads.shp")
gdf_traffic_p = gpd.read_file("shapes/traffic.shp")
gdf_transport_p = gpd.read_file("shapes/transport.shp")
gdf_waterways_p = gpd.read_file("shapes/waterways.shp")

In [29]:
#Create one dataframe including all information
df_all = pd.concat([gdf_building_a, gdf_landuse_a, gdf_natural_a, gdf_pofw_a, gdf_pois_a, gdf_traffic_a, gdf_water_a])

In [30]:
#Create gdf
gdf_all = gpd.GeoDataFrame(df_all, geometry='geometry')

In [31]:
gdf_all = gdf_all[['id','boundary', 'name', 'type', 'osm_id', 'fclass', 'name_2', 'type_2', 'geometry']]

In [32]:
gdf_all.head()

,id,boundary,name,type,osm_id,fclass,name_2,type_2,geometry
0,relation/2192363,administrative,Københavns Kommune,boundary,4250592,building,Rigshospitalet,hospital,"POLYGON ((12.56522 55.69626, 12.56602 55.69578..."
1,relation/2192363,administrative,Københavns Kommune,boundary,14309996,building,Nykredit,office,"POLYGON ((12.57572 55.67008, 12.57634 55.66973..."
2,relation/2192363,administrative,Københavns Kommune,boundary,14310073,building,Copenhagen Marriott Hotel,NaN,"POLYGON ((12.57381 55.66875, 12.57379 55.66876..."
3,relation/2192363,administrative,Københavns Kommune,boundary,23540747,building,NaN,apartments,"POLYGON ((12.53987 55.69390, 12.53987 55.69393..."
4,relation/2192363,administrative,Københavns Kommune,boundary,24507644,building,Valby Hallen,NaN,"POLYGON ((12.51590 55.64946, 12.51725 55.64942..."


### Buildings

In [33]:
gdf_buildings = gdf_building_a[['id', 'fclass', 'geometry']]

In [34]:
#Set matching CRS to the street gdf
gdf_buildings = gdf_buildings.to_crs("EPSG:32632")

In [36]:
#Find intersections between buildings and road polygons
spatial_join_buildings = gpd.sjoin(gdf_buildings, gdf_roads_short, how = 'inner', predicate = "intersects")

In [37]:
spatial_join_buildings

,id,fclass,geometry,index_right,osmid,u,v
0,relation/2192363,building,"POLYGON ((724043.171 6178036.111, 724096.116 6...",44734,216231104,2380873315,2380873319
0,relation/2192363,building,"POLYGON ((724043.171 6178036.111, 724096.116 6...",44743,216231104,2380873319,2380873315
0,relation/2192363,building,"POLYGON ((724043.171 6178036.111, 724096.116 6...",82545,"[216231104, 229497550]",8390857281,2380873319
0,relation/2192363,building,"POLYGON ((724043.171 6178036.111, 724096.116 6...",44742,"[216231104, 229497550]",2380873319,8390857281
0,relation/2192363,building,"POLYGON ((724043.171 6178036.111, 724096.116 6...",44741,229497548,2380873319,2380873318
...,...,...,...,...,...,...,...
64120,relation/2192363,building,"POLYGON ((725371.479 6177722.980, 725377.368 6...",99891,1145134056,10657723413,10657723420
64118,relation/2192363,building,"POLYGON ((725353.515 6177737.829, 725359.523 6...",99900,1145134056,10657723420,10657723413
64120,relation/2192363,building,"POLYGON ((725371.479 6177722.980, 725377.368 6...",99900,1145134056,10657723420,10657723413
64119,relation/2192363,building,"POLYGON ((725366.000 6177726.131, 725364.582 6...",99893,"[1145134056, 1145134059]",10657723413,10657723417


In [38]:
#Count buildings for each street by counting osmids, and create new dataframe
buildings_count = (spatial_join_buildings.groupby(["u", "v"])["osmid"].count().to_frame("buildings_count"))

buildings_count.head()

buildings_count
u      v                          
118744 1277939654                1
       1277939659                2
125401 527857404                 3
125423 6465638890                6
725111 2398732086                1

In [39]:
#Count values
buildings_count.value_counts()

buildings_count
1                  3757
2                   163
3                    15
4                    12
6                     2
5                     1
Name: count, dtype: int64

In [40]:
#Save count of buildings to a csv-file
#buildings_count.to_csv("spatial_features/building_count.csv", header = True)

These steps are repeated for all categories of spatial footprint features

### Landuse

In [42]:
gdf_landuse = gdf_landuse_a[['id', 'fclass', 'geometry']]

In [44]:
gdf_landuse = gdf_landuse.to_crs("EPSG:32632")

In [45]:
spatial_join_landuse = gpd.sjoin(gdf_landuse, gdf_roads_short, how = 'inner', predicate = "intersects")

In [46]:
spatial_join_landuse.shape

(72848, 7)

In [47]:
#Count landuse for each street by counting osmids, and create new dataframe
landuse_count = (spatial_join_landuse.groupby(["u", "v"])["osmid"].count().to_frame("landuse_count"))

landuse_count.head()

landuse_count
u      v                        
118781 1555492641              1
118783 118784                  1
       1555492641              1
       1555492647              1
118784 118783                  1

In [49]:
#landuse_count.to_csv("spatial_features/landuse_count.csv", header = True)

### Natural

In [50]:
### Natural
gdf_natural = gdf_natural_a[['id', 'fclass', 'geometry']]

In [51]:
gdf_natural = gdf_natural.to_crs("EPSG:32632")

In [53]:
spatial_join_natural = gpd.sjoin(gdf_natural, gdf_roads_short, how = 'inner', predicate = "intersects")

In [55]:
#Count natural for each street by counting osmids, and create new dataframe
natural_count = (spatial_join_natural.groupby(["u", "v"])["osmid"].count().to_frame("natural_count"))

natural_count.head()

natural_count
u         v                        
247089750 247089752               1
          247089895               1
          1339877008              1
          8499210013              1
247089752 247089750               1

In [57]:
#natural_count.to_csv("spatial_features/natural_count.csv", header = True)

### pofw

In [60]:
gdf_pofw = gdf_pofw_a[['id', 'fclass', 'geometry']]

In [61]:
gdf_pofw = gdf_pofw.to_crs("EPSG:32632")

In [62]:
spatial_join_pofw = gpd.sjoin(gdf_pofw, gdf_roads_short, how = 'inner', predicate = "intersects")

In [63]:
spatial_join_pofw.shape

(26, 7)

In [64]:
#Count pofw for each street by counting osmids, and create new dataframe
pofw_count = (spatial_join_pofw.groupby(["u", "v"])["osmid"].count().to_frame("pofw_count"))

pofw_count

,,pofw_count
u,v,
257790024,9725124545,1
279992717,411631105,1
411631105,279992717,1
1574092463,10839839397,1
2380873318,2380873319,1
2380873319,2380873318,1
3188612087,3188612089,1
3188612089,3188612087,1
3468170567,3468170568,1


In [65]:
#pofw_count.to_csv("spatial_features/pofw_count.csv", header = True)

### pois

In [67]:
gdf_pois_a.shape

(2601, 32)

In [68]:
gdf_pois = gdf_pois_a[['id', 'fclass', 'geometry']]

In [69]:
gdf_pois = gdf_pois.to_crs("EPSG:32632")

In [70]:
gdf_pois.crs

<Derived Projected CRS: EPSG:32632>
Name: WGS 84 / UTM zone 32N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 6°E and 12°E, northern hemisphere between equator and 84°N, onshore and offshore. Algeria. Austria. Cameroon. Denmark. Equatorial Guinea. France. Gabon. Germany. Italy. Libya. Liechtenstein. Monaco. Netherlands. Niger. Nigeria. Norway. Sao Tome and Principe. Svalbard. Sweden. Switzerland. Tunisia. Vatican City State.
- bounds: (6.0, 0.0, 12.0, 84.0)
Coordinate Operation:
- name: UTM zone 32N
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [71]:
spatial_join_pois = gpd.sjoin(gdf_pois, gdf_roads_short, how = 'inner', predicate = "intersects")

In [73]:
#Count pois for each street by counting osmids, and create new dataframe
pois_count = (spatial_join_pois.groupby(["u", "v"])["osmid"].count().to_frame("pois_count"))

pois_count

pois_count
u           v                      
118744      1277939654            1
            1277939659            2
125404      3093669085            1
654737      523637766             1
697549      298548451             1
...                             ...
11039574318 5320228682            1
11039661886 5320228662            1
11050942375 7508678338            1
11050942382 10851398896           1
11061276798 10063887225           1

[23107 rows x 1 columns]

In [74]:
pois_count.value_counts()

pois_count
1             19304
2              2858
3               841
4                88
6                16
Name: count, dtype: int64

In [75]:
#pois_count.to_csv("spatial_features/pois_count.csv", header = True)

### Traffic

In [76]:
gdf_traffic_a.crs
gdf_traffic_a.shape

(12, 32)

In [77]:
gdf_traffic = gdf_traffic_a[['id', 'fclass', 'geometry']]

In [78]:
gdf_traffic = gdf_traffic.to_crs("EPSG:32632")

In [79]:
spatial_join_traffic = gpd.sjoin(gdf_traffic, gdf_roads_short, how = 'inner', predicate = "intersects")

In [80]:
spatial_join_traffic.shape

(11, 7)

In [81]:
traffic_count = (spatial_join_traffic.groupby(["u", "v"])["osmid"].count().to_frame("traffic_count"))
traffic_count

,,traffic_count
u,v,
1063570532,1063570545,1
1063570545,1063570532,1
1181499794,9837951695,1
1387249555,2698667370,1
1387405746,7596712870,1
2380873316,2824284164,1
2516856968,1181499794,1
2698667370,1387249555,1
2824284164,2380873316,1


In [82]:
traffic_count.value_counts()

traffic_count
1                11
Name: count, dtype: int64

In [83]:
#traffic_count.to_csv("spatial_features/traffic_count.csv", header = True)

### Water

In [84]:
### Water
gdf_water_a.crs
gdf_water_a.shape

(265, 32)

In [85]:
gdf_water = gdf_water_a[['id', 'fclass', 'geometry']]

In [86]:
gdf_water = gdf_water.to_crs("EPSG:32632")

In [87]:
spatial_join_water = gpd.sjoin(gdf_water, gdf_roads_short, how = 'inner', predicate = "intersects")

In [88]:
spatial_join_water.shape

(313, 7)

In [89]:
water_count = (spatial_join_water.groupby(["u", "v"])["osmid"].count().to_frame("water_count"))
water_count

water_count
u           v                       
118795      107456766              1
            1302474338             1
118811      450789549              1
1055829     1055862                1
1055839     1055848                1
...                              ...
11047148066 11047098643            1
11047190428 11047190431            1
11047190431 11047190428            1
11047190437 11047190615            1
11047190615 11047190437            1

[305 rows x 1 columns]

In [90]:
water_count.value_counts()

water_count
1              297
2                8
Name: count, dtype: int64

In [91]:
#water_count.to_csv("spatial_features/water_count.csv", header = True)

In [95]:
#points
gdf_natural = gpd.read_file("shapes/natural.shp")
gdf_places = gpd.read_file("shapes/places.shp")
gdf_pofw = gpd.read_file("shapes/pofw.shp")
gdf_pois = gpd.read_file("shapes/pois.shp")
gdf_railways = gpd.read_file("shapes/railways.shp")
gdf_roads = gpd.read_file("shapes/roads.shp")
gdf_traffic = gpd.read_file("shapes/traffic.shp")
gdf_transport = gpd.read_file("shapes/transport.shp")
gdf_waterways = gpd.read_file("shapes/waterways.shp")

### Natural points

In [96]:
gdf_natural_p.crs
gdf_natural_p.shape

(15452, 3)

In [97]:
gdf_natural_p = gdf_natural_p[['id', 'fclass', 'geometry']]

In [98]:
gdf_natural_p = gdf_natural_p.to_crs("EPSG:32632")

In [99]:
spatial_join_natural_p = gpd.sjoin(gdf_natural_p, gdf_roads_short, how = 'inner', predicate = "intersects")

In [100]:
spatial_join_natural_p.shape

(12, 7)

In [101]:
natural_p_count = (spatial_join_natural_p.groupby(["u", "v"])["osmid"].count().to_frame("natural_p_count"))
natural_p_count

,,natural_p_count
u,v,
298719910,1802824731,1
1255233390,1305669021,1
1305669021,1255233390,1
1584360551,1587010807,1
1587010807,1584360551,1
1802824731,298719910,1
6402189094,7424040085,1
7424040085,6402189094,1
7719030124,7719030125,1


In [102]:
natural_p_count.value_counts()

natural_p_count
1                  12
Name: count, dtype: int64

In [103]:
#natural_p_count.to_csv("spatial_features/natural_p_count.csv", header = True)

### Places

In [104]:
gdf_places_p.crs
gdf_places_p.shape

(39, 33)

In [105]:
gdf_places_p = gdf_places_p[['id', 'fclass', 'geometry']]

In [106]:
gdf_places_p = gdf_places_p.to_crs("EPSG:32632")

In [107]:
spatial_join_places_p = gpd.sjoin(gdf_places_p, gdf_roads_short, how = 'inner', predicate = "intersects")

In [108]:
spatial_join_places_p.shape

(0, 7)

In [109]:
places_p_count = (spatial_join_places_p.groupby(["u", "v"])["osmid"].count().to_frame("places_p_count"))
places_p_count

,,places_p_count
u,v,


In [110]:
places_p_count.value_counts()

Series([], Name: count, dtype: int64)

In [111]:
#places_p_count.to_csv("spatial_features/places_p_count.csv", header = True)

There are no instances of this on the map,  therefore I do not save it. 

### pofw points

In [112]:
gdf_pofw_p.crs
gdf_pofw_p.shape

(12, 32)

In [113]:
gdf_pofw_p = gdf_pofw_p[['id', 'fclass', 'geometry']]

In [114]:
gdf_pofw_p = gdf_pofw_p.to_crs("EPSG:32632")

In [115]:
spatial_join_pofw_p = gpd.sjoin(gdf_pofw_p, gdf_roads_short, how = 'inner', predicate = "intersects")

In [116]:
spatial_join_pofw_p.shape

(0, 7)

In [117]:
pofw_p_count = (spatial_join_pofw_p.groupby(["u", "v"])["osmid"].count().to_frame("pofw_p_count"))
pofw_p_count

,,pofw_p_count
u,v,


In [118]:
pofw_p_count.value_counts()

Series([], Name: count, dtype: int64)

In [119]:
#pofw_p_count.to_csv("spatial_features/pofw_p_count.csv", header = True)

There are no instances of this, therefore I do not save it. 

### pois points


In [120]:
gdf_pois_p.crs
gdf_pois_p.shape

(12325, 32)

In [121]:
gdf_pois_p = gdf_pois_p[['id', 'fclass', 'geometry']]

In [122]:
gdf_pois_p = gdf_pois_p.to_crs("EPSG:32632")

In [123]:
spatial_join_pois_p = gpd.sjoin(gdf_pois_p, gdf_roads_short, how = 'inner', predicate = "intersects")

In [124]:
spatial_join_pois_p.shape

(46, 7)

In [125]:
pois_p_count = (spatial_join_pois_p.groupby(["u", "v"])["osmid"].count().to_frame("pois_p_count"))
pois_p_count

pois_p_count
u           v                        
118744      1277939659              1
283712721   1063636565              1
297073114   297073157               1
297073157   297073114               1
1063636540  1063636565              1
1063636565  283712721               1
            1063636540              1
            1350224661              1
1271881177  6357644328              1
1277939659  118744                  1
1350224661  1063636565              1
1622854710  9924993841              1
2120239980  9924993838              1
2631272263  4965953355              1
2700982467  5712456765              1
3635266134  3635266135              1
3635266135  3635266134              1
4237583607  4237583608              1
4237583608  4237583607              1
4965953355  2631272263              1
5009606823  9129880670              1
5712456765  2700982467              1
6218150597  6856686378              1
6270574508  6270574510              1
6270574510  6270574508              1
6856686366  6856686378              1
6856686374  6856686378              1
6856686375  6856686378              1
6856686378  6218150597              1
            6856686366              1
            6856686374              1
            6856686375              1
6943466042  6943466043              1
6943466043  6943466042              1
7726386285  1271881177              1
9129880670  5009606823              1
9711796148  9711796147              1
9718509404  9718509398              1
9924993838  2120239980              1
9924993841  1622854710              1
9924993844  9924993845              1
9924993845  9924993844              1
9966107145  9966107151              1
9966107151  9966107145              1
10015689502 10015689511             1
10015689511 10015689502             1

In [126]:
pois_p_count.value_counts()

pois_p_count
1               46
Name: count, dtype: int64

In [127]:
#pois_p_count.to_csv("spatial_features/pois_p_count.csv", header = True)

### railways


In [128]:
gdf_railways_p.crs
gdf_railways_p.shape

(1054, 35)

In [129]:
gdf_railways_p = gdf_railways_p[['id', 'fclass', 'geometry']]

In [130]:
gdf_railways_p = gdf_railways_p.to_crs("EPSG:32632")

In [131]:
spatial_join_railways_p = gpd.sjoin(gdf_railways_p, gdf_roads_short, how = 'inner', predicate = "intersects")

In [132]:
spatial_join_railways_p.shape

(4144, 7)

In [133]:
railways_p_count = (spatial_join_railways_p.groupby(["u", "v"])["osmid"].count().to_frame("railways_p_count"))
railways_p_count

railways_p_count
u           v                            
118725      6357644306                  4
118787      274394389                   2
118792      118806                      2
            1052708411                  2
118806      118792                      2
...                                   ...
10956822537 6469570343                  2
10964856176 1189253953                  2
11002100444 11002100463                 4
11002100463 11002100444                 4
11039263874 1281443548                  2

[2409 rows x 1 columns]

In [134]:
railways_p_count.value_counts()

railways_p_count
1                   1315
2                    848
4                    103
3                     74
5                     24
6                     13
8                      9
7                      6
10                     6
9                      5
15                     4
11                     2
Name: count, dtype: int64

In [135]:
#railways_p_count.to_csv("spatial_features/railways_p_count.csv", header = True)

### roads


In [136]:
gdf_roads_p.crs
gdf_roads_p.shape

(30414, 38)

In [137]:
gdf_roads_p = gdf_roads_p[['id', 'fclass', 'geometry']]

In [138]:
gdf_roads_p = gdf_roads_p.to_crs("EPSG:32632")

In [139]:
spatial_join_roads_p = gpd.sjoin(gdf_roads_p, gdf_roads_short, how = 'inner', predicate = "intersects")

In [140]:
spatial_join_roads_p.shape

(347489, 7)

In [141]:
roads_p_count = (spatial_join_roads_p.groupby(["u", "v"])["osmid"].count().to_frame("roads_p_count"))
roads_p_count

roads_p_count
u           v                         
118725      2512504197               3
            6357644306               4
118730      118751                   7
118732      283603631                6
            6534097568               6
...                                ...
11064633329 11064633331              1
            11064633335              1
11064633331 11064633329              1
11064633335 11064633328              1
            11064633329              1

[101981 rows x 1 columns]

In [142]:
roads_p_count.value_counts()

roads_p_count
3                54171
4                19220
2                12280
5                 8682
6                 3314
1                 1987
7                 1388
8                  498
9                  224
10                  92
11                  40
12                  34
14                  16
13                  15
15                   6
16                   4
17                   4
18                   2
20                   2
27                   2
Name: count, dtype: int64

In [143]:
#roads_p_count.to_csv("spatial_features/roads_p_count.csv", header = True)

### traffic


In [144]:
gdf_traffic_p.crs
gdf_traffic_p.shape

(5792, 32)

In [145]:
gdf_traffic_p = gdf_traffic_p[['id', 'fclass', 'geometry']]

In [146]:
gdf_traffic_p = gdf_traffic_p.to_crs("EPSG:32632")

In [147]:
spatial_join_traffic_p = gpd.sjoin(gdf_traffic_p, gdf_roads_short, how = 'inner', predicate = "intersects")

In [148]:
spatial_join_traffic_p.shape

(13976, 7)

In [149]:
traffic_p_count = (spatial_join_traffic_p.groupby(["u", "v"])["osmid"].count().to_frame("traffic_p_count"))
traffic_p_count

traffic_p_count
u           v                          
118725      2512504197                1
118730      118751                    1
118732      283603631                 2
            6534097568                1
118735      4393215585                1
...                                 ...
11039263874 1281443548                1
11042711819 4985131979                1
11059280905 1055200733                1
11059280939 1055200733                1
            6651812227                1

[10000 rows x 1 columns]

In [150]:
traffic_p_count.value_counts()

traffic_p_count
1                  6239
2                  3611
3                   109
4                    27
5                    10
7                     2
8                     2
Name: count, dtype: int64

In [151]:
#traffic_p_count.to_csv("spatial_features/traffic_p_count.csv", header = True)

### transport


In [152]:
gdf_transport_p.crs
gdf_transport_p.shape

(1374, 32)

In [153]:
gdf_transport_p = gdf_transport_p[['id', 'fclass', 'geometry']]

In [154]:
gdf_transport_p = gdf_transport_p.to_crs("EPSG:32632")

In [155]:
spatial_join_transport_p = gpd.sjoin(gdf_transport_p, gdf_roads_short, how = 'inner', predicate = "intersects")

In [156]:
spatial_join_transport_p.shape

(478, 7)

In [157]:
transport_p_count = (spatial_join_transport_p.groupby(["u", "v"])["osmid"].count().to_frame("transport_p_count"))
transport_p_count

,,transport_p_count
u,v,
118792,1052708411,1
576709,8579330032,1
697538,7224398,1
698030,2460384709,1
698663,5583563213,1
...,...,...
10048226646,8319476223,1
10075093698,8086438,1
10155079966,2207188959,1


In [158]:
transport_p_count.value_counts()

transport_p_count
1                    394
2                     39
3                      2
Name: count, dtype: int64

In [159]:
#transport_p_count.to_csv("spatial_features/transport_p_count.csv", header = True)

### waterways

In [160]:
gdf_waterways_p.crs
gdf_waterways_p.shape

(100, 33)

In [161]:
gdf_waterways_p = gdf_waterways_p[['id', 'fclass', 'geometry']]

In [162]:
gdf_waterways_p = gdf_waterways_p.to_crs("EPSG:32632")

In [163]:
spatial_join_waterways_p = gpd.sjoin(gdf_waterways_p, gdf_roads_short, how = 'inner', predicate = "intersects")

In [164]:
spatial_join_waterways_p.shape

(198, 7)

In [165]:
waterways_p_count = (spatial_join_waterways_p.groupby(["u", "v"])["osmid"].count().to_frame("waterways_p_count"))
waterways_p_count

,,waterways_p_count
u,v,
375469,298530864,1
698675,278783420,1
781544,298527246,1
1055762,560705649,1
1055809,560705653,1
...,...,...
10587036950,10587037002,1
10587037002,10587036950,1
10587037003,6018007909,1


In [166]:
waterways_p_count.value_counts()

waterways_p_count
1                    194
2                      2
Name: count, dtype: int64

In [167]:
#waterways_p_count.to_csv("spatial_features/waterways_p_count.csv", header = True)